<h2>Helper Notebook for Creating Code Flow Data</h2>

In [51]:
# REYeker lib
import modules.rEYEkerAnalysis as rEYEker

# lib for dataframes
import pandas as pd

import numpy as nt

# lib for plotting
from matplotlib import pyplot as plt

# lib for crerating paths
from pathlib import Path

import random

In [52]:
def draw_lines(sheet_config, width):
    height_of_line = sheet_config.loc[1]['stop'] - sheet_config.loc[1]['start']
    
    for idx, val in sheet_config.iterrows():
        plt.text(0, val['start']+height_of_line/2, str(idx))
        plt.plot([0, width], [val['start'], val['start']], color='grey')
        
def draw_heat(img, aoi_mask, aoi_colors):
    for height in range(img.shape[0]):
        for width in range(img.shape[1]):
            if aoi_mask[height, width] == 0:
                break
            img[height, width] = 0.4 * img[height, width] + 0.6 * aoi_colors[aoi_mask[height, width]]

<h3>Config</h3>

In [49]:
algo = "Swap"

image_path = 'images/' + algo + '.png'

#path for line sheet
sheet_path = './data/code_flow/'+ algo + '.xlsx'

#name for excel file saved in ./data/code_flow folder
save_as = 'AOI_' + algo

print(image_path)
print(sheet_path)
print(save_as)


images/Swap.png
data/code_flow/Swap.xlsx
AOI_Swap


In [50]:
sheet_config = pd.read_excel(sheet_path, sheet_name="config")


img = rEYEker.load_image(image_path)
aoi_mask = np.zeros((img.shape[0], img.shape[1])).astype(int)

# iterative: none, main, Iterative definition, Pre calculation, Iteration Condition, Iteration Step, Return Result
# recursive: none, main, Recursive definition, Pre calculation, Recusrive Condition, Recusrive Step, Return Result

aoi_regions = [
    #line, name
    (-1, "none"),
    
    (2, "main"),
    (3, "main"),
    (4, "main"),
    (5, "main"),
    
    
    (7, "Iterative definition"),
    
    (8, "Pre calculation"),
    (9, "Pre calculation"),
    
    (10, "Iteration Condition"),
    
    (11, "Iteration Step"),
    (12, "Iteration Step"),
    (13, "Iteration Step"),
    
    (15, "Return Result"),
    (16, "Return Result"),
]

aoi_names = list(dict.fromkeys([name for (line, name) in aoi_regions]))

for region in aoi_regions:
    if region[0] == -1:
        continue
    aoi_mask[sheet_config.loc[region[0]]['start']:sheet_config.loc[region[0]]['stop'], :] = aoi_names.index(region[1])

aoi_colors = [np.array([1.0, 1.0, 1.0])]

for i in range(np.amax(aoi_mask)):
    aoi_colors.append(np.array([random.uniform(0,1), random.uniform(0,1), random.uniform(0,1)]))  

draw_heat(img, aoi_mask, aoi_colors)
draw_lines(sheet_config, img.shape[1])
 

plt.imshow(img);

FileNotFoundError: [Errno 2] No such file or directory: 'data/code_flow/Swap.xlsx'

save Data to Excel

In [248]:
cols = ["startHeight", "stopHeight", "startWidth", "stopWidth", "Name"]
tmp_list = []
for region in aoi_regions:
    if region[0] == -1:
        continue
    tmp_list.append([sheet_config.loc[region[0]]['start'], sheet_config.loc[region[0]]['stop'],0, img.shape[1], region[1]])
    
df_config = pd.DataFrame(tmp_list, columns=cols)

folder = "./data/aoi_categorized/"
Path(folder).mkdir(parents=True, exist_ok=True)

writer = pd.ExcelWriter(folder + save_as + '.xlsx', engine='xlsxwriter')
df_config.to_excel(writer, index=False)

writer.save()